In [2]:
import os
import speech_recognition as sr

In [3]:
path1 = './videos/'
path2 = './audio/'

# Video file to Audio file

In [4]:
for filename in os.listdir(path1):
    print(filename)
    if (filename.endswith(".mp4")): #or .avi, .mpeg, whatever.
        id = os.path.splitext(filename)[0]
        os.system("ffmpeg -i {0} {1}.wav".format(os.path.join(path1,filename), os.path.join(path2,id)))
    else:
        continue

7123195527327698222.mp4


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 13.0.0 (clang-1300.0.29.30)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable

7081298765352537350.mp4
.DS_Store
7145293556553551150.mp4


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 13.0.0 (clang-1300.0.29.30)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable

7099531036937260330.mp4
7017540145020751106.mp4


...

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 13.0.0 (clang-1300.0.29.30)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable

7054746897738698031.mp4


size=    9812kB time=00:00:56.91 bitrate=1412.4kbits/s speed= 426x    
video:0kB audio:9812kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.000776%
ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 13.0.0 (clang-1300.0.29.30)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass

7086798266838322437.mp4
7115762593788071174.mp4


Input #0, mov,mp4,m4a,3gp,3g2,mj2, from './videos/7115762593788071174.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.45.100
  Duration: 00:00:16.02, start: 0.000000, bitrate: 1465 kb/s
  Stream #0:0[0x1](und): Video: h264 (High) (avc1 / 0x31637661), yuv420p(tv, bt709, progressive), 576x1024 [SAR 1:1 DAR 9:16], 1427 kb/s, 30.19 fps, 60 tbr, 15360 tbn (default)
    Metadata:
      handler_name    : VideoHandler
      vendor_id       : [0][0][0][0]
  Stream #0:1[0x2](und): Audio: aac (HE-AACv2) (mp4a / 0x6134706D), 44100 Hz, stereo, fltp, 32 kb/s (default)
    Metadata:
      handler_name    : SoundHandler
      vendor_id       : [0][0][0][0]
Stream mapping:
  Stream #0:1 -> #0:0 (aac (native) -> pcm_s16le (native))
Press [q] to stop, [?] for help
Output #0, wav, to './audio/7115762593788071174.wav':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2av

# Audio file to Text file

In [7]:
r = sr.Recognizer()
for filename in os.listdir(path2):
    if (filename.endswith(".wav")): #or .avi, .mpeg, whatever.
        audio = sr.AudioFile(os.path.join(path2,filename))
        with audio as source:
            audio = r.record(source)                  
            result = r.recognize_google(audio,language = 'en-US',show_all=True)
            print(filename)
            print(result['alternative'][0]['transcript'])
    else:
        continue

7054746897738698031.wav
there's a new foundation in town so we're going to try it in case you haven't heard Charlotte Tilbury made Hollywood flawless filter in a foundation form this is called Charlotte's beautiful skin Foundation since they're supposedly so similar we're going to compare them side by side starting off with flawless filter and shade three on this side of my face we just blended it in stunning now on to Charlotte's beautiful skin I got shade sex so I hope it matches all right blending it in let's see your kidding look how stunning okay so comparing the two obviously the foundation has more coverage than the flawless filter but they both have that really similar radiance which is shuts kiss let's try applying some Foundation over the flawless filter technically flawless filter is just a primer this side is a little bit more radiant than this side what do we think you guys
7145293556553551150.wav
this and this they're the same thing dupes


# Applying BERT sentiment analysis to extracted text

In [8]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures


/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-23 17:09:35.299753: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model.summary()

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [10]:
import tensorflow as tf

pred_sentences = ["there's a new foundation in town so we're going to try it in case you haven't heard Charlotte Tilbury made Hollywood flawless filter in a foundation form this is called Charlotte's beautiful skin Foundation since they're supposedly so similar we're going to compare them side by side starting off with flawless filter and shade three on this side of my face we just blended it in stunning now on to Charlotte's beautiful skin I got shade sex so I hope it matches all right blending it in let's see your kidding look how stunning okay so comparing the two obviously the foundation has more coverage than the flawless filter but they both have that really similar radiance which is shuts kiss let's try applying some Foundation over the flawless filter technically flawless filter is just a primer this side is a little bit more radiant than this side what do we think you guys","this and this they're the same thing dupes"]
tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['Negative','Positive']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
for i in range(len(pred_sentences)):
  print(pred_sentences[i], ": \n", labels[label[i]])

there's a new foundation in town so we're going to try it in case you haven't heard Charlotte Tilbury made Hollywood flawless filter in a foundation form this is called Charlotte's beautiful skin Foundation since they're supposedly so similar we're going to compare them side by side starting off with flawless filter and shade three on this side of my face we just blended it in stunning now on to Charlotte's beautiful skin I got shade sex so I hope it matches all right blending it in let's see your kidding look how stunning okay so comparing the two obviously the foundation has more coverage than the flawless filter but they both have that really similar radiance which is shuts kiss let's try applying some Foundation over the flawless filter technically flawless filter is just a primer this side is a little bit more radiant than this side what do we think you guys : 
 Positive
this and this they're the same thing dupes : 
 Positive
